In [1]:
import pandas as pd
import os
import asyncio
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.utils.cell import get_column_letter

In [2]:
async def read_excel_sheets(file_path: str) -> List[str]:
    """엑셀 파일의 모든 시트 이름을 반환합니다."""
    try:
        df = pd.ExcelFile(file_path)
        return df.sheet_names
    except Exception as e:
        print(f"엑셀 시트 읽기 오류: {str(e)}")
        raise ValueError(f"엑셀 파일 읽기 실패: {str(e)}")

In [3]:
def _get_cell_color(cell) -> Optional[str]:
        """
        셀의 배경색 RGB 값을 반환
        
        Args:
            cell: openpyxl 셀 객체
            
        Returns:
            RGB 문자열 또는 None (배경색이 없는 경우)
        """
        if cell.fill.patternType == 'solid':
            return cell.font.color.rgb
        return None

In [21]:
file_path = "/appdata/storage/research/original/2. EIU_AllDataByGeography_로데이터.xlsx"
CODES = ["PSBR", "DCPI", "CARA", "BALC", "XRPD", "XPP1", "XPP2", "XPP3", "XPP4", "FRES", "MEXP", "MIMP", "MPP1","MPP2", "MPP3","PUDP","DGDP","TDPY","BALM"]
HEADER = ["Country_Code","Series", "Code", "Currency", "Units"]

In [23]:

async def process_data(file_path:str) :

    print(f"원본 파일 처리 시작 : {file_path}")

    workbook = openpyxl.load_workbook(file_path)
    sheet_names = workbook.sheetnames

    # 원본 데이터와 색상 정보를 저장할 리스트
    row_datas = []

    for sheet_name in sheet_names:
        print(f"시트 처리중 : {sheet_name}")
        sheet = workbook[sheet_name]

        header_row = None

        # 헤더 행 찾기 (보통 'Series'와 'Code' 컬럼이 있는 행)
        for row_idx in range(1,20): # 상위 20행 내에서 헤더 찾기기
            if sheet.cell(row=row_idx, column=1).value == "Series" and sheet.cell(row=row_idx, column=2).value == "Code":
                header_row = row_idx
                break

        if header_row is None:
            print(f"헤더 행을 찾을 수 없습니다. : {sheet_name}")
            return Exception(f"헤더 행을 찾을 수 없습니다. : {sheet_name}")
                    
        # print(f"헤더 행 찾음 : {header_row}")

        # 컬럼 이름 추출
        column_names = []
        col_idx = 1
        while True :
            cell_value = sheet.cell(row=header_row, column=col_idx).value
            if cell_value is None :
                break
            column_names.append(cell_value)
            col_idx += 1

        print(f"열 이름 : {column_names}")

        #연도 컬럼 식별
        year_columns = [col for col in column_names if isinstance(col,str) and col.strip().isdigit()]

        # 시트의 데이터를 코드별로 저장할 딕셔너리
        sheet_data_by_code = {}

        # 데이터 행 처리
        for row_idx in range(header_row + 1, sheet.max_row + 1):
            row_data = {}
            
            # 국가 코드 추가
            row_data["Country_Code"] = sheet_name

            for col_idx, col_name in enumerate(column_names, start=1):
                
                cell = sheet.cell(row=row_idx, column=col_idx)
                cell_value = cell.value

                if col_name == "Code" :
                    code_value = cell_value

                if isinstance(col_name, str) and col_name.strip().isdigit():
                    color = _get_cell_color(cell)
                    if cell_value != "–" :
                        if color == "0000588D" : # 블랙
                            cell_value = f"EST|{cell_value}"
                        else :
                            cell_value = f"ACT|{cell_value}"

                # print(f"{col_name} 행 {row_idx} 열 {col_idx} 값 : {cell_value}")

                row_data[col_name] = cell_value

            if code_value and code_value in CODES :
                sheet_data_by_code[code_value] = row_data

    
        for code in CODES :
            if code in sheet_data_by_code :
                row_datas.append(sheet_data_by_code[code])
            else :
                default_row = {
                    "Country_Code":sheet_name,
                    "Code": code,
                    "Series": "",
                    "Currency": "",
                    "Units": "",
                    "Source": "",
                    "Definition": "",
                    "Note": "",
                    "Published": ""
                }

                for year in year_columns:
                    default_row[year] = "-"

                row_datas.append(default_row)

    df = pd.DataFrame(row_datas)

    if df.empty :
        raise ValueError("처할 수 있는 데이터가 없습니다.")
    
    df = _transform_data(data=df,year_columns=year_columns)
    
    return df                 

In [41]:
def _transform_data(data:pd.DataFrame,year_columns:List[str]) -> pd.DataFrame :
    """
        원본 데이터를 목표 형식으로 변환
        
        Args:
            data: 원본 데이터프레임
            
        Returns:
            변환된 데이터프레임
    """
    # # 데이터 변환을 위한 빈 데이터프레임 생성
    # transformed_data = pd.DataFrame()

    # #국가 명을 eiu_cont_en_nm 컬럼으로 변환
    # transformed_data['eiu_cont_en_nm'] = data['Country_Code']

    # #국가 코드를 eiu_country_code 컬럼으로 변환
    # transformed_data['eiu_country_code'] = data['Country_Code']

    # transformed_data['eiu_series_title'] = data['Series']

    # #
    # transformed_data['eiu_currency'] = data['Currency']
    df:pd.DataFrame = data[HEADER+year_columns]
    columns = {
        # "Country_Code":"eiu_cont_en_nm",
        "Country_Code":"eiu_country_code",
        "Series":"eiu_series_title",
        "Currency":"eiu_currency",
        "Code":"eiu_code",
        "Units":"eiu_units"
    }


    if year_columns :

        min_year = min(map(int,year_columns)) 
        max_year = max(map(int,year_columns))

        max_YY = max_year % 100 # 연도 두자리

        for year in range(min_year,max_year+1) :
            columns[f"{year}"] = f"eiu_year{year%100}"

        for i in range(max_YY+1, 52) :
            df[f"eiu_year{i}"] = "FOR"
            
    df.rename(columns=columns,inplace=True)

    return df

In [42]:
df = await process_data(file_path)
df[df["eiu_country_code"]=="CA"]

원본 파일 처리 시작 : /appdata/storage/research/original/2. EIU_AllDataByGeography_로데이터.xlsx
시트 처리중 : BE
열 이름 : ['Series', 'Code', 'Currency', 'Units', 'Source', 'Definition', 'Note', 'Published', '2019', '2020', '2021', '2022', '2023']
시트 처리중 : CA
열 이름 : ['Series', 'Code', 'Currency', 'Units', 'Source', 'Definition', 'Note', 'Published', '2019', '2020', '2021', '2022', '2023']
시트 처리중 : CN
열 이름 : ['Series', 'Code', 'Currency', 'Units', 'Source', 'Definition', 'Note', 'Published', '2019', '2020', '2021', '2022', '2023']
시트 처리중 : CO
열 이름 : ['Series', 'Code', 'Currency', 'Units', 'Source', 'Definition', 'Note', 'Published', '2019', '2020', '2021', '2022', '2023']
시트 처리중 : EG
열 이름 : ['Series', 'Code', 'Currency', 'Units', 'Source', 'Definition', 'Note', 'Published', '2019', '2020', '2021', '2022', '2023']
시트 처리중 : DE
열 이름 : ['Series', 'Code', 'Currency', 'Units', 'Source', 'Definition', 'Note', 'Published', '2019', '2020', '2021', '2022', '2023']
시트 처리중 : JP
열 이름 : ['Series', 'Code', 'Currency', '

/tmp/ipykernel_2022/1456205666.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"eiu_year{i}"] = "FOR"
/tmp/ipykernel_2022/1456205666.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"eiu_year{i}"] = "FOR"
/tmp/ipykernel_2022/1456205666.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

,eiu_country_code,eiu_series_title,eiu_code,eiu_currency,eiu_units,eiu_year19,eiu_year20,eiu_year21,eiu_year22,eiu_year23,...,eiu_year42,eiu_year43,eiu_year44,eiu_year45,eiu_year46,eiu_year47,eiu_year48,eiu_year49,eiu_year50,eiu_year51
19,CA,Budget balance (% of GDP),PSBR,,,ACT|-0.017,ACT|-10.949,ACT|-2.919,ACT|0.109,ACT|-0.594,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
20,CA,Consumer prices (% change pa; av),DCPI,,,EST|1.9,EST|0.7,EST|3.4,EST|6.8,EST|3.9,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
21,CA,Current-account balance (% of GDP),CARA,,,EST|-2,EST|-2,EST|0,ACT|-0.353,EST|-0.7,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
22,CA,Current-account balance (US$),BALC,$,m,EST|-34000,EST|-33300,EST|300,ACT|-7622,EST|-15600,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
23,CA,Exchange rate LCU:US$ (av),XRPD,,,ACT|1.326842,ACT|1.341358,ACT|1.253583,ACT|1.301725,ACT|1.349175,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
24,CA,Export 1 (% share),XPP1,,,ACT|19.656,ACT|14.333,ACT|21.172,ACT|27.224,–,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
25,CA,Export 2 (% share),XPP2,,,ACT|11.644,ACT|13.328,ACT|12.44,ACT|11.382,–,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
26,CA,Export 3 (% share),XPP3,,,ACT|10.819,ACT|12.716,ACT|12.031,ACT|10.994,–,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
27,CA,Export 4 (% share),XPP4,,,ACT|15.819,ACT|14.505,ACT|11.302,ACT|10.396,–,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
28,CA,Foreign-exchange reserves (US$),FRES,$,m,ACT|85297,ACT|90428,ACT|106615,ACT|106952,ACT|117551,...,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR,FOR
